# Capítulo 4 ISLP

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

c:\Users\Mariajose\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
weekly = pd.read_csv("../db/Weekly.csv")
weekly.head(3)

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up


In [4]:
eda = ProfileReport(weekly, title="EDA")
eda.to_file("../db/eda_weekly.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 74.78it/s]


In [5]:
# Use the full data set to perform a logistic regression
X = weekly[["Lag1", "Lag2", "Lag3", "Lag4", "Lag5", "Volume"]]
y = weekly["Direction"]

model = LogisticRegression()
model.fit(X, y)
print(f"Los coeficientes son {model.coef_}")
print(f"El intercepto es {model.intercept_}")

Los coeficientes son [[-0.04123352  0.05839182 -0.01604123 -0.02777908 -0.01445306 -0.02273943]]
El intercepto es [0.26685095]


In [6]:
# Ver significancia de los coeficientes
matriz_cov = model.coef_.T @model.coef_
errores_estandar = np.sqrt(np.diag(matriz_cov))
z_score = 1.96
c_inferior = model.coef_ - z_score * errores_estandar
c_superior = model.coef_ + z_score * errores_estandar

df_intervalos = pd.DataFrame({"Coeficiente": model.coef_.flatten(), 
                              "Límite inferior": c_inferior.flatten(),
                              "Límite superior": c_superior.flatten()},
                              index=X.columns)

df_intervalos

,Coeficiente,Límite inferior,Límite superior
Lag1,-0.041234,-0.122051,0.039584
Lag2,0.058392,-0.056056,0.172840
Lag3,-0.016041,-0.047482,0.015400
Lag4,-0.027779,-0.082226,0.026668
Lag5,-0.014453,-0.042781,0.013875
Volume,-0.022739,-0.067309,0.021830


In [7]:
# Predecir valores para evaluar el funcionamiento del modelo
y_pred = model.predict(X)

# calcular errores
exactitud = accuracy_score(y, y_pred, normalize=True)
exactitud

0.5610651974288338

In [11]:
matriz_confusion = confusion_matrix(y, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(matriz_confusion, annot=True, fmt="d", cmap="Blues", xticklabels=["Down", "Up"], yticklabels=["Down", "Up"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
print(matriz_confusion)

[[ 54 430]
 [ 48 557]]


C:\Users\Mariajose\AppData\Local\Temp\ipykernel_14844\4156367735.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Observamos que el modelo presenta un sesgo hacia clasificador "todo" como un positivo. Dicho de otra manera, tenemos principalmente error del tipo II.